# Домашняя работа по теме "Машинное обучение ранжированию"

В этом ДЗ мы:
- научимся работать со стандартным датасетом для машинного обучения ранжированию [MSLR](https://www.microsoft.com/en-us/research/project/mslr/)
- попробуем применить на практике все то, чему мы научились на семинаре

## Как будет происходить сдача ДЗ

Вам надо:
- форкнуть эту репу
- создать бранч в котором вы дальше будете работать
- реализовать класс Model в этом ноутбуке
- убедиться, что ваша реализация выбивает NDCG@10 выше бейзлайна (см. ниже)
- запушить ваш бранч и поставить Pull Request
- в комментарии написать какой скор вы выбили

В таком случае мы (организаторы):

- счекаутим вашу бранчу
- проверим что ваша реализация действительно выбивает заявленный скор

Предполагается, что и вы, и мы работаем в виртаульном окружении как в семинаре про машинное обучение ранжированию: seminars/7-learning-to-rank/requirements.txt(подробнее про работу с виртуальными окружениями README в корне этой репы).

Оценка:
- За выбитый скор больше **0.507** назначаем **5** баллов, за скор больше (или равно) **0.510** назначаем максимальный балл -- 10 баллов
- Тот из участников кто выбъет самый высокий скор получит еще +10 баллов

При сдаче кода важно помнить о том, что:
- В коде не должно быть захардкоженных с потолка взятых гиперпараметров (таких как число деревьев, learning rate и т.п.) -- обязательно должен быть представлен код который их подбирает!
- Решение должно быть стабильно от запуска к запуску (на CPU) т.е. все seed'ы для генераторов случайных чисел должны быть фиксированы
- Мы (организаторы) будем запускать код на CPU поэтому, даже если вы использовали для подбора параметров GPU, финальный скор надо репортить на CPU

## Пререквизиты

Импортируем все что нам понадобится для дальнейшей работы:

In [1]:
import pathlib
from timeit import default_timer as timer
import copy

import numpy as np
import pandas as pd

import catboost
from catboost import datasets, utils

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

Загрузим датасет MSLR.

Полный датасет можно скачать с официального сайта: https://www.microsoft.com/en-us/research/project/mslr/

Строго говоря, он состоит их 2х частей:

- основной датасет MSLR-WEB30K -- он содержит более 30 тыс. запросов
- "маленький" датасет MSLR-WEB10K, который содержит только 10 тыс. запросов и является случайным сэмплом датасета MSLR-WEB30K

в этом ДЗ мы будем работать с MSLR-WEB10K, т.к. полная версия датасета может просто не поместиться у нас в RAM (и, тем более, в память видеокарты если мы учимся на GPU)

Будем считать, что мы самостоятельно скачали датасет MSLR-WEB10K с официального сайта, поместили его в папку КОРЕНЬ-ЭТОЙ-РЕПЫ/data/mslr-web10k и раззиповали.

В результате у нас должна получиться следующая структура папок:

In [3]:
!ls ../../data

Fold1  Fold2  Fold3  Fold4  Fold5  MSLR-WEB10K


In [6]:
!ls -lh ../../data/MSLR-WEB10K/

итого 20K
drwxrwxr-x 2 james james 4,0K апр 28  2010 Fold1
drwxrwxr-x 2 james james 4,0K апр 28  2010 Fold2
drwxrwxr-x 2 james james 4,0K апр 28  2010 Fold3
drwxrwxr-x 2 james james 4,0K апр 28  2010 Fold4
drwxrwxr-x 2 james james 4,0K апр 28  2010 Fold5


Заметим, что датасет довольно большой, в распакованном виде он весит 7.7 GB.

Датасет состоит из нескольких фолдов, которые по сути представляют из себя разные разбиения одних и тех же данных на обучающее, валидационное и тестовые множеста.

Дальше мы будем использовать только первый фолд: Fold1.

Заглянем внутрь:

In [7]:
!ls -lh ../../data/MSLR-WEB10K/Fold1

итого 1,3G
-rw-rw-r-- 1 james james 267M апр 30  2010 test.txt
-rw-rw-r-- 1 james james 800M апр 30  2010 train.txt
-rw-rw-r-- 1 james james 261M апр 30  2010 vali.txt


Видим, что у нас 3 файла с говорящими названиями, соответсвующими сплитам нашего датасета.

Посмотрим на содержимое одного из файлов:

In [8]:
!head -n 1 ../../data/MSLR-WEB10K/Fold1/train.txt

2 qid:1 1:3 2:3 3:0 4:0 5:3 6:1 7:1 8:0 9:0 10:1 11:156 12:4 13:0 14:7 15:167 16:6.931275 17:22.076928 18:19.673353 19:22.255383 20:6.926551 21:3 22:3 23:0 24:0 25:6 26:1 27:1 28:0 29:0 30:2 31:1 32:1 33:0 34:0 35:2 36:1 37:1 38:0 39:0 40:2 41:0 42:0 43:0 44:0 45:0 46:0.019231 47:0.75000 48:0 49:0 50:0.035928 51:0.00641 52:0.25000 53:0 54:0 55:0.011976 56:0.00641 57:0.25000 58:0 59:0 60:0.011976 61:0.00641 62:0.25000 63:0 64:0 65:0.011976 66:0 67:0 68:0 69:0 70:0 71:6.931275 72:22.076928 73:0 74:0 75:13.853103 76:1.152128 77:5.99246 78:0 79:0 80:2.297197 81:3.078917 82:8.517343 83:0 84:0 85:6.156595 86:2.310425 87:7.358976 88:0 89:0 90:4.617701 91:0.694726 92:1.084169 93:0 94:0 95:2.78795 96:1 97:1 98:0 99:0 100:1 101:1 102:1 103:0 104:0 105:1 106:12.941469 107:20.59276 108:0 109:0 110:16.766961 111:-18.567793 112:-7.760072 113:-20.838749 114:-25.436074 115:-14.518523 116:-21.710022 117:-21.339609 118:-24.497864 119:-27.690319 120:-20.203779 121:-15.449379 122:-4.474452 123:-23.634899 

Видим, что данные лежат в уже знакомом нам по семинару формате:

- В первой колонке лежит таргет (оценка асессора), по 5-балльной шкале релевантности: от 0 до 4 (включительно)
- Во второй колонке лежит ID запроса, по которому можно сгруппировать все оценки документов в рамках одного и того же запроса
- Дальше идет вектор из 128 фичей (таких как значения BM25 и т.п.), их точная природа нам сейчас на важна

В файле qid и все-фичи кодируются в формате КЛЮЧ:ЗНАЧЕНИЕ, напр. 130:116 -- тут 130 это номер фичи, а 116 -- ее значение.

Такой формат в мире машинного обучения часто называют svm light формат (в честь когда-то популярной библиотеки SVM-Light)

Напишем немного вспомогательного кода для загрузки этого датасета:

In [9]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns
    
def load_svmlight_file(input_file, max_num_lines=0):
    """Loads dataset split in SVM-Light format"""
    def _parse_field(field):
        parts = field.split(':')
        if len(parts) != 2:
            raise Exception(f"invalid number of parts in field {field}")
        return parts

    num_features = 136
    exp_num_fields = num_features + 2
    num_lines = 0
    X = []
    with open(input_file, 'rt') as f:
        for line in f:
            try:
                num_lines += 1
                                  
                # Parse into fields
                fields = line.rstrip().split(' ')
                num_fields = len(fields)
                if num_fields != exp_num_fields:
                    raise Exception(f"invalid number of fields {num_fields}")
    
                # Parse every field
                x = np.zeros(exp_num_fields, dtype=np.float32)
                label = int(fields[0])
                x[0] = label
                _, qid_str = _parse_field(fields[1])
                qid = int(qid_str)
                x[1] = qid
                for i, field in enumerate(fields[2:]):
                    _, feature_str = _parse_field(field)
                    x[i+2] = float(feature_str)
    
                # Add new object
                X.append(x)
                if num_lines % 50000 == 0:
                    print(f"Loaded {num_lines} lines...")
                if max_num_lines > 0 and num_lines == max_num_lines:
                    print(f"WARNING: stop loading, line limit reached: max_num_lines = {max_num_lines} input_file = {input_file}")
                    break
            except Exception as e:
                raise Exception(f"error at line {num_lines} in {input_file}") from e
    
    # To pandas
    df = pd.DataFrame(X, columns=generate_column_names(num_features))
    print(f"Loaded SVM-Light file {input_file}")
    return df

И теперь загрузим датасет:

In [10]:
fold_dir = pathlib.Path("../../data/MSLR-WEB10K/Fold1")

df_train = load_svmlight_file(fold_dir.joinpath("train.txt"))
df_valid = load_svmlight_file(fold_dir.joinpath("vali.txt"))
df_test = load_svmlight_file(fold_dir.joinpath("test.txt"))
print(f"Dataset loaded from fold_dir {fold_dir}")

Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded 250000 lines...
Loaded 300000 lines...
Loaded 350000 lines...
Loaded 400000 lines...
Loaded 450000 lines...
Loaded 500000 lines...
Loaded 550000 lines...
Loaded 600000 lines...
Loaded 650000 lines...
Loaded 700000 lines...
Loaded SVM-Light file ../../data/MSLR-WEB10K/Fold1/train.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded SVM-Light file ../../data/MSLR-WEB10K/Fold1/vali.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded SVM-Light file ../../data/MSLR-WEB10K/Fold1/test.txt
Dataset loaded from fold_dir ../../data/MSLR-WEB10K/Fold1


Посмотрим на данные:

In [11]:
print(df_train.head(5))

   label  qid  feature_1  feature_2  feature_3  feature_4  feature_5  \
0    2.0  1.0        3.0        3.0        0.0        0.0        3.0   
1    2.0  1.0        3.0        0.0        3.0        0.0        3.0   
2    0.0  1.0        3.0        0.0        2.0        0.0        3.0   
3    2.0  1.0        3.0        0.0        3.0        0.0        3.0   
4    1.0  1.0        3.0        0.0        3.0        0.0        3.0   

   feature_6  feature_7  feature_8  ...  feature_127  feature_128  \
0        1.0        1.0   0.000000  ...         62.0   11089534.0   
1        1.0        0.0   1.000000  ...         54.0   11089534.0   
2        1.0        0.0   0.666667  ...         45.0          3.0   
3        1.0        0.0   1.000000  ...         56.0   11089534.0   
4        1.0        0.0   1.000000  ...         64.0          5.0   

   feature_129  feature_130  feature_131  feature_132  feature_133  \
0          2.0        116.0      64034.0         13.0          3.0   
1          2

Т.е. теперь мы видим что данные доступны в точно таком же виде, как это было в семинаре.

Проведем небольшой EDA.

Всего у нас 723 тыс. документов в трейне:

In [12]:
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723412 entries, 0 to 723411
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 380.8 MB
None


235 тыс. документов в валидации:

In [13]:
print(df_valid.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235259 entries, 0 to 235258
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 123.8 MB
None


И 241 тыс. документов в тесте:

In [14]:
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241521 entries, 0 to 241520
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 127.1 MB
None


Сколько у нас запросов?

In [15]:
num_queries_train = df_train['qid'].nunique()
num_queries_valid = df_valid['qid'].nunique()
num_queries_test = df_test['qid'].nunique()
print(f"Got {num_queries_train} train, {num_queries_valid} valid and {num_queries_test} test queries")

Got 6000 train, 2000 valid and 2000 test queries


In [16]:
def to_catboost_dataset(df):
    # check_list = df['qid'].value_counts()

    # for indx, values in check_list.items():
    #     if values > 1023:
    #         print(indx, values)

    #         indxs = df[df['qid'] == indx].index
    #         to_delete = values - 1023

    #         for i in range(to_delete):
    #             print(i)
    #             df = df.drop(indxs[i])
    #     else:
    #         break

    y = df["label"].to_numpy()  # Label: [0-4]
    q = df["qid"].to_numpy().astype("uint32")  # Query Id
    X = df.drop(columns=["label", "qid"]).to_numpy()  # 136 features
    return (X, y, q)


X_train, y_train, q_train = to_catboost_dataset(df_train)
X_valid, y_valid, q_valid = to_catboost_dataset(df_valid)
X_test, y_test, q_test = to_catboost_dataset(df_test)

In [17]:
pool_train = catboost.Pool(data=X_train, label=y_train, group_id=q_train)
pool_valid = catboost.Pool(data=X_valid, label=y_valid, group_id=q_valid)
pool_test = catboost.Pool(data=X_test, label=y_test, group_id=q_test)

In [18]:
EVAL_METRIC = "NDCG:top=10;type=Exp"

In [19]:
DEFAULT_PARAMS = {
    "iterations": 6000,  # maximum possible number of trees
    "early_stopping_rounds": 100,  # stop if metric does not improve for N rounds
    "eval_metric": EVAL_METRIC,  # # metric used for early stopping
    "random_seed": 22,
    "verbose": 10,
    'task_type': 'GPU',
}

## Обучаем модель

Теперь можно приступить непосредственно к обучению модели. 

Объявим класс модели, который надо будем заимлементить в этом ДЗ:

In [20]:
class Model:
    def __init__(self, model_params):
        params = copy.deepcopy(model_params)

        loss_function = "YetiRank"
        # Temporary directory that is used by catboost to store additional information
        catboost_info_dir = f"/tmp/catboost_info.{loss_function.lower()}"

        params.update(
            {
                "loss_function": loss_function,
                "train_dir": str(catboost_info_dir),
            }
        )

        self.model = catboost.CatBoost(params)

    def fit(self, pool_train, pool_valid):
        self.model.fit(pool_train, eval_set=pool_valid, use_best_model=True)

    def predict(self, pool_test):
        return self.model.predict(pool_test)

In [51]:
import random

def grid_search(iters, pool_train, pool_valid):
    np.random.seed(0)
    random.seed(0)
    best_score = 0
    best_model = None
    best_params = {}
    y_valid, q_valid = pool_valid.get_label(), pool_valid.get_group_id_hash()

    for i in range(iters):
        params = {
            "learning_rate": np.random.uniform(1e-3, 1e-1),
            "depth": random.randint(3, 8),
            "l2_leaf_reg": np.random.uniform(1, 50),
            "loss_function": "YetiRank",
            "eval_metric": "NDCG:top=10;type=Exp",
            "iterations": 6000,
            "bagging_temperature": np.random.uniform(0.7, 1),
            "use_best_model": True,
            "min_data_in_leaf": random.randint(1, 200),
            "verbose": 200,
            "grow_policy": random.choice(["SymmetricTree", "Depthwise", "Lossguide"]),
            "task_type": "GPU",
        }

        model = catboost.CatBoost(params)
        model.fit(pool_train, eval_set=pool_valid)
        score = utils.eval_metric(
            y_valid, model.predict(pool_valid), "NDCG:top=10;type=Exp", group_id=q_valid
        )[0]
        
        if score > best_score:
            best_model = model
            best_score = score
            best_params = params

    return best_model, best_params

In [52]:
best_model, best_params = grid_search(100, pool_train, pool_valid)

-------------------- Model number 1 --------------------
Params:
	 - learning_rate: 0.055332536888805156
	 - depth: 6
	 - l2_leaf_reg: 21.740491624800164
	 - loss_function: YetiRank
	 - eval_metric: NDCG:top=10;type=Exp
	 - iterations: 6000
	 - bagging_temperature: 0.8808290128214932
	 - use_best_model: True
	 - min_data_in_leaf: 98
	 - verbose: 200
	 - grow_policy: Depthwise
	 - task_type: GPU


Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3395152	best: 0.3395152 (0)	total: 501ms	remaining: 50m 7s
200:	test: 0.4798628	best: 0.4798628 (200)	total: 7.2s	remaining: 3m 27s
400:	test: 0.4931687	best: 0.4931687 (400)	total: 13.8s	remaining: 3m 12s
600:	test: 0.4992109	best: 0.4992109 (600)	total: 20.4s	remaining: 3m 3s
800:	test: 0.5024551	best: 0.5024551 (800)	total: 26.9s	remaining: 2m 54s
1000:	test: 0.5043150	best: 0.5043150 (1000)	total: 33.3s	remaining: 2m 46s
1200:	test: 0.5053389	best: 0.5053389 (1200)	total: 39.7s	remaining: 2m 38s
1400:	test: 0.5066487	best: 0.5066487 (1400)	total: 46s	remaining: 2m 30s
1600:	test: 0.5073600	best: 0.5073894 (1585)	total: 52.3s	remaining: 2m 23s
1800:	test: 0.5083051	best: 0.5083051 (1800)	total: 58.6s	remaining: 2m 16s
2000:	test: 0.5091862	best: 0.5091862 (2000)	total: 1m 4s	remaining: 2m 9s
2200:	test: 0.5100482	best: 0.5100518 (2185)	total: 1m 11s	remaining: 2m 2s
2400:	test: 0.5104858	best: 0.5106101 (2390)	total: 1m 17s	remaining: 1m 55s
2600:	test: 0.5110430	best: 0.

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.2717074	best: 0.2717074 (0)	total: 61.6ms	remaining: 6m 9s
200:	test: 0.4513446	best: 0.4513446 (200)	total: 5.4s	remaining: 2m 35s
400:	test: 0.4691936	best: 0.4691936 (400)	total: 10.8s	remaining: 2m 30s
600:	test: 0.4768727	best: 0.4768727 (600)	total: 16.1s	remaining: 2m 24s
800:	test: 0.4811149	best: 0.4811891 (795)	total: 21.4s	remaining: 2m 19s
1000:	test: 0.4834580	best: 0.4834580 (1000)	total: 26.7s	remaining: 2m 13s
1200:	test: 0.4855392	best: 0.4855392 (1200)	total: 32.1s	remaining: 2m 8s
1400:	test: 0.4874110	best: 0.4874341 (1395)	total: 37.3s	remaining: 2m 2s
1600:	test: 0.4888003	best: 0.4888003 (1600)	total: 42.7s	remaining: 1m 57s
1800:	test: 0.4900141	best: 0.4900141 (1800)	total: 48s	remaining: 1m 51s
2000:	test: 0.4910767	best: 0.4910767 (2000)	total: 53.3s	remaining: 1m 46s
2200:	test: 0.4925476	best: 0.4925476 (2200)	total: 58.6s	remaining: 1m 41s
2400:	test: 0.4932932	best: 0.4932932 (2400)	total: 1m 3s	remaining: 1m 35s
2600:	test: 0.4944175	best: 0.4

...

0:	test: 0.3706138	best: 0.3706138 (0)	total: 86.3ms	remaining: 8m 37s
200:	test: 0.4708049	best: 0.4708600 (195)	total: 8.72s	remaining: 4m 11s
400:	test: 0.4790146	best: 0.4790146 (400)	total: 17.3s	remaining: 4m 1s
600:	test: 0.4852728	best: 0.4852728 (600)	total: 25.9s	remaining: 3m 53s
800:	test: 0.4893789	best: 0.4893789 (800)	total: 34.4s	remaining: 3m 43s
1000:	test: 0.4925756	best: 0.4925756 (1000)	total: 42.8s	remaining: 3m 33s
1200:	test: 0.4956087	best: 0.4956267 (1195)	total: 51.2s	remaining: 3m 24s
1400:	test: 0.4980375	best: 0.4980375 (1400)	total: 59.5s	remaining: 3m 15s
1600:	test: 0.4999201	best: 0.4999707 (1585)	total: 1m 7s	remaining: 3m 6s
1800:	test: 0.5015952	best: 0.5015952 (1800)	total: 1m 16s	remaining: 2m 57s
2000:	test: 0.5029347	best: 0.5030237 (1970)	total: 1m 24s	remaining: 2m 48s
2200:	test: 0.5039998	best: 0.5039998 (2200)	total: 1m 32s	remaining: 2m 39s
2400:	test: 0.5051702	best: 0.5051702 (2400)	total: 1m 40s	remaining: 2m 31s
2600:	test: 0.5059919	b

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3266255	best: 0.3266255 (0)	total: 79.5ms	remaining: 7m 57s
200:	test: 0.4679116	best: 0.4679116 (200)	total: 7.31s	remaining: 3m 31s
400:	test: 0.4827012	best: 0.4827012 (400)	total: 14.6s	remaining: 3m 23s
600:	test: 0.4897136	best: 0.4897136 (600)	total: 21.8s	remaining: 3m 16s
800:	test: 0.4941441	best: 0.4941441 (800)	total: 29.1s	remaining: 3m 8s
1000:	test: 0.4971496	best: 0.4971496 (1000)	total: 36.4s	remaining: 3m 2s
1200:	test: 0.4996130	best: 0.4996130 (1200)	total: 43.8s	remaining: 2m 54s
1400:	test: 0.5018776	best: 0.5018968 (1395)	total: 51.1s	remaining: 2m 47s
1600:	test: 0.5035809	best: 0.5035809 (1600)	total: 58.4s	remaining: 2m 40s
1800:	test: 0.5047612	best: 0.5047912 (1795)	total: 1m 5s	remaining: 2m 33s
2000:	test: 0.5059894	best: 0.5059894 (2000)	total: 1m 12s	remaining: 2m 25s
2200:	test: 0.5066438	best: 0.5066768 (2180)	total: 1m 20s	remaining: 2m 18s
2400:	test: 0.5075498	best: 0.5075679 (2390)	total: 1m 27s	remaining: 2m 11s
2600:	test: 0.5086929	be

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.2377083	best: 0.2377083 (0)	total: 64.7ms	remaining: 6m 27s
200:	test: 0.4580340	best: 0.4580340 (200)	total: 5.79s	remaining: 2m 47s
400:	test: 0.4721013	best: 0.4721013 (400)	total: 11.5s	remaining: 2m 40s
600:	test: 0.4795204	best: 0.4795204 (600)	total: 17.2s	remaining: 2m 34s
800:	test: 0.4835206	best: 0.4835414 (795)	total: 22.9s	remaining: 2m 28s
1000:	test: 0.4866122	best: 0.4866122 (1000)	total: 28.6s	remaining: 2m 22s
1200:	test: 0.4891151	best: 0.4891151 (1200)	total: 34.3s	remaining: 2m 17s
1400:	test: 0.4908921	best: 0.4908921 (1400)	total: 40s	remaining: 2m 11s
1600:	test: 0.4925974	best: 0.4925974 (1600)	total: 45.6s	remaining: 2m 5s
1800:	test: 0.4938368	best: 0.4938368 (1800)	total: 51.3s	remaining: 1m 59s
2000:	test: 0.4947563	best: 0.4947563 (2000)	total: 57s	remaining: 1m 53s
2200:	test: 0.4961970	best: 0.4962878 (2185)	total: 1m 2s	remaining: 1m 48s
2400:	test: 0.4969666	best: 0.4970107 (2385)	total: 1m 8s	remaining: 1m 42s
2600:	test: 0.4980062	best: 0.

Создадим и применим модель:

In [51]:
best_params = {
    "learning_rate": 0.06216979249983486,
    "depth": 8,
    "l2_leaf_reg": 1.3838688750080852,
    "loss_function": "YetiRank",
    "eval_metric": "NDCG:top=10;type=Exp",
    "iterations": 10000,
    "bagging_temperature": 0.8041700553796658,
    "use_best_model": True,
    "min_data_in_leaf": 58,
    "verbose": 200,
    "grow_policy": "Depthwise",
    "task_type": "GPU",
    'random_seed': 43,
}

In [52]:
# Create model
model = Model(best_params)

# Fit
start = timer()

model.fit(pool_train, pool_valid)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f}")

# Predict
y_hat_test = model.predict(pool_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.3677108	best: 0.3677108 (0)	total: 33.1ms	remaining: 5m 31s
200:	test: 0.4961553	best: 0.4961553 (200)	total: 3.64s	remaining: 2m 57s
400:	test: 0.5058468	best: 0.5058664 (395)	total: 7.19s	remaining: 2m 52s
600:	test: 0.5095624	best: 0.5097157 (595)	total: 10.5s	remaining: 2m 44s
800:	test: 0.5120766	best: 0.5121867 (795)	total: 13.8s	remaining: 2m 38s
1000:	test: 0.5135945	best: 0.5135945 (1000)	total: 17s	remaining: 2m 32s
1200:	test: 0.5141642	best: 0.5143172 (1165)	total: 20.2s	remaining: 2m 27s
1400:	test: 0.5149137	best: 0.5149852 (1360)	total: 23.3s	remaining: 2m 23s
1600:	test: 0.5155672	best: 0.5157052 (1545)	total: 26.5s	remaining: 2m 18s
1800:	test: 0.5159030	best: 0.5159159 (1730)	total: 29.7s	remaining: 2m 15s
2000:	test: 0.5166323	best: 0.5167031 (1860)	total: 32.8s	remaining: 2m 11s
2200:	test: 0.5163564	best: 0.5167359 (2015)	total: 35.9s	remaining: 2m 7s
2400:	test: 0.5164290	best: 0.5167359 (2015)	total: 39s	remaining: 2m 3s
2600:	test: 0.5163912	best: 0.5

Теперь, имея предикты, можно посчитать метрики качества:

In [54]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=10;type=Exp']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")

# Get test targets and groups
y_test = df_test['label'].to_numpy()
q_test = df_test['qid'].to_numpy().astype('uint32')
    
# Compute metrics on test
score = compute_metrics(y_test, y_hat_test, q_test)

metric = NDCG:top=10;type=Exp score = 0.512


CPU fit

In [ ]:
best_params["task_type"] = 'CPU'

model = Model(best_params)

# Fit
start = timer()

model.fit(pool_train, pool_valid)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f}")

# Predict
y_hat_test = model.predict(pool_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

compute_metrics(y_test, y_hat_test, q_test)

Save model params

In [55]:
import pickle

with open('params.pkl', 'wb') as f:
    pickle.dump(best_params, f)

Ожидаем, что ваша модель покажет результаты выше бейзлайна!